In [1]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from gw_signal_tools.matrix_with_units import MatrixWithUnits

In [2]:
test = MatrixWithUnits([42.0], u.s)

print(np.sqrt(test))
print(test**(1/2))

[6.4807407]
array([<Quantity 6.4807407 s(1/2)>], dtype=object)


In [3]:
test = MatrixWithUnits(np.array([[42, 96], [96, 42]]), u.s)

np.linalg.inv(test)

array([[-0.00563607,  0.01288245],
       [ 0.01288245, -0.00563607]])

In [4]:
np.linalg.inv(np.array([[42, 96], [96, 42]]) * u.s)

<Quantity [[-0.00563607,  0.01288245],
           [ 0.01288245, -0.00563607]] 1 / s>

In [5]:
test = np.linalg.inv(np.array([[42 * u.s, 96 * u.m], [96 * u.m, 42 * u.s]]))

TypeError: only dimensionless scalar quantities can be converted to Python scalars

In [10]:
# test = MatrixWithUnits(np.array([[42, 96], [96, 42]]), np.array([[u.s, u.m], [u.m, u.s]], dtype=object))
test = MatrixWithUnits(np.array([[42, 96], [96, 42]]), np.array([[1.0 * u.s**2, 1.0 * u.s * u.m], [1.0 * u.m * u.s, 1.0 * u.m**2]], dtype=object))
# test = MatrixWithUnits(np.array([[1, 0], [0, 1]]), np.array([[1.0 * u.s**2, 1.0 * u.s * u.m], [1.0 * u.m * u.s, 1.0 * u.m**2]], dtype=object))

test_inv = MatrixWithUnits(np.linalg.inv(test), test.unit**-1)
# test_inv = MatrixWithUnits(np.linalg.inv(test), 1.0 / test.unit)

print(test)
print(test_inv)

print(test.value @ test_inv.value)
print(test @ test_inv)

array([[<Quantity 42. s2>, <Quantity 96. m s>],
       [<Quantity 96. m s>, <Quantity 42. m2>]], dtype=object)
array([[<Quantity -0.00563607 1 / s2>, <Quantity 0.01288245 1 / (m s)>],
       [<Quantity 0.01288245 1 / (m s)>, <Quantity -0.00563607 1 / m2>]],
      dtype=object)
[[ 1.00000000e+00 -2.77555756e-17]
 [ 3.81639165e-17  1.00000000e+00]]
array([[1.0, -2.7755575615628914e-17],
       [3.8163916471489756e-17, 0.9999999999999998]], dtype=object)


In [12]:
type(np.array([[1.0 * u.s**2, 1.0 * u.s * u.m], [1.0 * u.m * u.s, 1.0 * u.m**2]], dtype=object)[0,0])

astropy.units.quantity.Quantity

In [ ]:
u.Quantity(1.0 * test_inv.unit[0,0])

<Quantity 1. 1 / s2>

In [14]:
print(1.0 * u.s / u.m + 1.0 * u.s / u.m)  # No error
# print(u.s / u.m + u.s / u.m)  # Error

2.0 s / m


Crazy...

## Tests with Units and Matrix

In [2]:
test = np.zeros((2, 2), dtype=object)
# test = np.zeros((2, 2), dtype=u.Quantity)
# test = np.array([0, 0], dtype=object)
# test = u.Quantity(np.zeros((2, 2), dtype=object), dtype=object)

test[0, 0] = 1.0 * u.s
test[0, 1] = 1.0 * u.Msun
test[1, 0] = 1.0 * u.kg
test[1, 1] = 1.0 * u.m

print(test)

[[<Quantity 1. s> <Quantity 1. solMass>]
 [<Quantity 1. kg> <Quantity 1. m>]]


In [3]:
test_inv = np.zeros((2, 2), dtype=object)

test_inv[0, 0] = 1.0 / u.s
test_inv[0, 1] = 1.0 / u.Msun
test_inv[1, 0] = 1.0 / u.kg
test_inv[1, 1] = 1.0 / u.m

print(test * test_inv)

[[<Quantity 1.> <Quantity 1.>]
 [<Quantity 1.> <Quantity 1.>]]


In [4]:
values = np.full((2, 2), 42, dtype=float)
product = values * test
# product = np.array(values * test, dtype=u.Quantity)

print(product)
print(product.dtype)
print(product.astype(u.Quantity).dtype)
print(product.astype(u.Quantity).to_value())
print(product.astype(float))

[[<Quantity 42. s> <Quantity 42. solMass>]
 [<Quantity 42. kg> <Quantity 42. m>]]
object
object


AttributeError: 'numpy.ndarray' object has no attribute 'to_value'

In [5]:
np.linalg.inv(test)
np.linalg.inv(np.array(test, dtype=float))

UFuncTypeError: Cannot cast ufunc 'inv' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [6]:
test.astype(u.Quantity)
print(test.astype(u.Quantity))
print(test.astype(float))
print(u.Quantity(test, dtype=float).value)
print(u.Quantity(test, dtype=object).value)
print(u.Quantity(test.astype(u.Quantity)))
print(u.Quantity(test.astype(u.Quantity)).value)

[[<Quantity 1. s> <Quantity 1. solMass>]
 [<Quantity 1. kg> <Quantity 1. m>]]


TypeError: only dimensionless scalar quantities can be converted to Python scalars

In [7]:
with u.set_enabled_equivalencies(u.dimensionless_angles()):
    print(u.equivalencies.dimensionless_angles())
    # print(u.equivalencies.)
    # print(u.Quantity(test, dtype=float).value)

with u.set_enabled_units([u.m]):
    print(u.m.find_equivalent_units())
    # print(u.Quantity(test, dtype=float).value)

[(Unit("rad"), None)]
  Primary name | Unit definition | Aliases
[
  m            | irreducible     | meter   ,
]


## General Class Tests

GWPy attributes

In [4]:
from gwpy.frequencyseries import FrequencySeries

test = FrequencySeries([1])

test.epoch = 0.0

test.dx = 1.0 * u.Hz

print(test._epoch, test._dx)

test._dx = -1.0 * u.Hz

print(test._dx)

0 1.0 Hz
-1.0 Hz


Aha, once they are called, one can also access them

## Simple MatrixWithUnits Tests

In [2]:
test = MatrixWithUnits(np.array(42), u.s)
# test = MatrixWithUnits([2, 2], [u.s, u.m])

In [3]:
print(test)

# float(test)
print(np.array(test))
print(np.array(test).__array__())
print(type(np.array(test).__array__()))

<Quantity 42. s>
42
42
<class 'numpy.ndarray'>


In [4]:
float(np.array([1.0], dtype=object))
# float(np.array([1.0 * u.s], dtype=object))

/tmp/ipykernel_551851/3182476769.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  float(np.array([1.0], dtype=object))


1.0

In [5]:
class Fisher:
    def __init__(self, with_units=True):
        self._fisher = 42
        self._with_units = with_units
    
    # def fisher(self, with_units=False):
    #     if with_units:
    #         return self._fisher * u.s
    #     else:
    #         return self._fisher
    
    # @property
    # def fisher(self):
    #     self.fisher(self._with_units)
    

    # @property is cool because no fisher(...) needed and setters/getters
    # are custom. On the other hand, units thing is not convenient...
    # -> maybe make separate properties? One fisher and one fisher_with_units?
    @property
    def fisher(self):
        if self._with_units:
            return self._fisher
        else:
            return 96

    @property
    def return_with_units(self):
        return self._with_units
    
    @return_with_units.setter
    def return_with_units(self, value):
        if value != True and value != False:
            raise ValueError()
        else:
            self._with_units = value

In [6]:
from gwpy.frequencyseries import FrequencySeries

test = FrequencySeries([1])

list(test.xspan)

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


[0.0, 1.0]

In [7]:
test = Fisher()

print(test.fisher)

test._with_units = False

print(test.fisher)


42
96


Nice Idea: have value and unit properties

In [8]:
class FisherMatrix:
    def __new__(cls):
        ...

        # return self._value * self._unit


    @property
    def value(self):
        ...
        # return self._value
    

    @property
    def unit(self):
        ...
        # return self._unit

    @classmethod
    def plot(self):
        ...
        # plot with color=uncertainty


Hmmm, inverse as property is not convenient... Maybe make base class MatrixWithUnit and then have Fisher + its inverse being instances of that?

## MatrixWithUnits Tests

In [9]:
isinstance(u.m, u.IrreducibleUnit)

True

In [10]:
type(u.m**-1)

astropy.units.core.CompositeUnit

In [11]:
for val in np.reshape(42, -1):
    print(val)
    print(type(val))
    print(isinstance(val, (int, float, complex, np.number)))

42
<class 'numpy.int64'>
True


In [12]:
isinstance(2.0 * u.m, u.Quantity)

# u.Quantity(u.m)
type(u.m)


astropy.units.core.IrreducibleUnit

In [13]:
test = MatrixWithUnits(42, u.s)

print(test)
print(type(test))
print(test.value)
print(test.unit)

<Quantity 42. s>
<class 'gw_signal_tools.matrix_with_units.MatrixWithUnits'>
42.0
s


In [14]:
np.shape(42)

()

In [15]:
test_mat = np.ones((2, 2))
test_unit = np.array([[u.s, u.m], [u.m, u.kg]])

test_mat_unit = MatrixWithUnits(test_mat, test_unit)

In [16]:
test_mat_unit * test_mat_unit
test_mat_unit * 2.0
2.0 * test_mat_unit

array([[<Quantity 2. s>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 2. kg>]], dtype=object)

In [17]:
1.0/test_unit

array([[<Quantity 1. 1 / s>, <Quantity 1. 1 / m>],
       [<Quantity 1. 1 / m>, <Quantity 1. 1 / kg>]],
      dtype=object)

In [18]:
mult_test_1 = test_mat_unit * 2.0
mult_test_2 = test_mat_unit * u.m
# mult_test_3 = test_unit * u.m
mult_test_4 = np.asarray(u.m, dtype=object) * test_unit
mult_test_5 = test_mat_unit * test_mat_unit
mult_test_6 = 1.0 / test_mat_unit
mult_test_7 = test_mat_unit / 1.0
mult_test_8 = test_mat_unit / test_mat_unit
mult_test_9 = test_mat_unit**2
# mult_test_10 = 2.0 * u.m * test_mat_unit
mult_test_11 = test_mat_unit * 2.0 * u.m

print(mult_test_1)
print(mult_test_1.value)
print(mult_test_1.unit)

print(mult_test_2)
print(mult_test_2.value)
print(mult_test_2.unit)
# print(mult_test_3)
print(mult_test_4)
print(mult_test_5)
print(mult_test_6)
print(mult_test_7)
print(mult_test_8)
print(mult_test_9)

array([[<Quantity 2. s>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 2. kg>]], dtype=object)
[[2. 2.]
 [2. 2.]]
[[Unit("s") Unit("m")]
 [Unit("m") Unit("kg")]]
array([[<Quantity 1. m s>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg m>]], dtype=object)
[[1. 1.]
 [1. 1.]]
[[Unit("m s") Unit("m2")]
 [Unit("m2") Unit("kg m")]]
[[Unit("m s") Unit("m2")]
 [Unit("m2") Unit("kg m")]]
array([[<Quantity 1. s2>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg2>]], dtype=object)
array([[<Quantity 1. 1 / s>, <Quantity 1. 1 / m>],
       [<Quantity 1. 1 / m>, <Quantity 1. 1 / kg>]],
      dtype=object)
array([[<Quantity 1. s>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
array([[<Quantity 1.>, <Quantity 1.>],
       [<Quantity 1.>, <Quantity 1.>]], dtype=object)
array([[<Quantity 1. s2>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg2>]], dtype=object)


In [19]:
test_mat_unit = MatrixWithUnits(test_mat, test_unit)
print(test_mat_unit)

print(test_mat_unit[0, 0])  # Test getter
test_mat_unit[0, 0] = 42  # Test setter

print(test_mat_unit)

test_mat_unit[1, 1] = 96 * u.pc  # Another setter test

print(test_mat_unit)


# test_mat_unit[0, 0] = u.m  # Error, good

# print(test_mat_unit)

array([[<Quantity 1. s>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
1.0 s
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)


In [20]:
test = MatrixWithUnits(42, u.s)
print(type(test))

test.value = 96

print(test)

test.unit = u.m

print(test)


# print(float(test))

# print(float(test_mat_unit))

<class 'gw_signal_tools.matrix_with_units.MatrixWithUnits'>
<Quantity 96. s>
<Quantity 96. m>


In [21]:
test_arr = np.array([[u.s, u.m]], dtype=u.core.IrreducibleUnit)

test_arr[0, 0]

Unit("s")

In [22]:
test_copy = test_mat_unit.copy()

test_copy[0, 0] = -1 * u.kg

print(test_copy)
print(test_mat_unit)  # Unchanged, good. If we set equal without copy, then it also changes, so things seem to work

array([[<Quantity -1. kg>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)


In [23]:
print(isinstance(2.0 * u.m, u.Quantity))
print(isinstance(2.0 * u.m, (int, float, complex, np.number)))

True
False


In [24]:
test = MatrixWithUnits([[42, 96]], [[u.m, u.m]])
test = MatrixWithUnits(np.array([[42, 96]]), u.m)
# test = MatrixWithUnits(42, u.m)
print(type(test))
print(np.array(test))
# test_addition_6 = 2.0 * u.m + test

<class 'gw_signal_tools.matrix_with_units.MatrixWithUnits'>
[[42 96]]


In [25]:
test_addition_1 = test_mat_unit + 1.0
test_addition_2 = 1.0 + test_mat_unit
# test_addition_3 = test_mat_unit + u.m  # Should throw error, as it does
# test_addition_4 = test_mat_unit + test_mat_unit.unit  # Should throw error
test_addition_5 = test_mat_unit + test_mat_unit

test = MatrixWithUnits(42, u.m)
test = MatrixWithUnits([[42, 96]], [[u.m, u.m]])
# test = MatrixWithUnits(np.array([[42, 96]]), u.m)
# test_addition_6 = 2.0 * u.m + test
test_addition_7 = test + 2.0 * u.m
# test_addition_8 = test + 2.0 * u.s  # Incompatible unit, should throw error, as it does

print(test_addition_1)
print(test_addition_2)
print(test_addition_5)
# print(test_addition_6)
print(test_addition_7)

array([[<Quantity 43.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 97. pc>]], dtype=object)
array([[<Quantity 43.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 97. pc>]], dtype=object)
array([[<Quantity 84.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 192. pc>]], dtype=object)
array([[<Quantity 44. m>, <Quantity 98. m>]], dtype=object)


In [26]:
isinstance(np.array([42]), np.ndarray)

True

In [27]:
test.unit == u.m

np.equal(test.unit, u.m)

array([[ True,  True]])

In [28]:
test = MatrixWithUnits([[42, 96]], u.m)

print(test)

array([[<Quantity 42. m>, <Quantity 96. m>]], dtype=object)


In [29]:
class MatrixWithUnits_simple:
    _allowed_numeric_types = (int, float, complex, np.number)
    _allowed_unit_types = (u.IrreducibleUnit, u.CompositeUnit, u.Unit)


    def  __init__(self, value, units) -> None:
        # assert np.shape(value) == np.shape(unit), \
        #         ('`value` and `unit` must have equal shape if unit is an '
        #         'array of astropy units.')
        

        # Idea: add support for given unit that is "scalar"
        # -> should we also make sure value is np.array?

        # value = np.asarray(value, dtype=np.number)
        # value = np.asarray(value, dtype=float)

        if isinstance(units, self._allowed_unit_types):
            if not isinstance(value, self._allowed_numeric_types):
                unit = np.full(np.shape(value), units, dtype=object)
        else:
            assert np.shape(value) == np.shape(units), \
                ('`value` and `unit` must have equal shape if unit is an '
                'array of astropy units.')
            
        if not isinstance(value, (np.ndarray, u.Quantity)):
            value = np.asarray(value, dtype=float)

        if not isinstance(units, (np.ndarray, u.Quantity)):
            units = np.asarray(units, dtype=object)

        self.value = value
        self.units = units
        # NOTE: setting "private" properties here already is not good practice.
        # Instead go through setters of attributes, where these are set
        

    # def __new__(cls, value, unit):
    #     cls._value = value
    #     cls._unit = unit

    #     """
    #     TODOS
    #     - make sure they have same shape
    #     - maybe make sure unit has dtype object? Or that all members are of type u.Quantity?
    #     """
        
    #     return cls._value * cls._unit


    @property
    def value(self):
        ...
        return self._value
    
    @value.setter
    def value(self, value):
        try:
            assert np.shape(value) == np.shape(self.value), \
                'New and old `value` must have equal shape'
        except AttributeError:
            pass  # New class instance is created, nothing to check
            
        for val in np.reshape(value, -1):
            assert (isinstance(val, self._allowed_numeric_types)
                    and not isinstance(val, bool)), \
                f'Need valid numeric types for all members of `value` (not {type(val)}).'
        
        self._value = value


    @property
    def units(self):
        ...
        return self._units
    
    @units.setter
    def units(self, units):
        try:
            assert np.shape(units) == np.shape(self.unit), \
                'New and old `unit` must have equal shape'
        except AttributeError:
            pass  # New class instance is created, nothing to check
        
        for val in np.reshape(units, -1):
            assert isinstance(val, self._allowed_unit_types), \
                f'Need valid unit types for all members of `unit` (not {type(val)}).'
        
        self._units = units


    def __repr__(self) -> str:
        # return (self.value * self.unit).__repr__()
        return (np.array(self.value) * np.array(self.units)).__repr__()
    

    def __add__(self, other):
        return MatrixWithUnits(self.value + other.value, self.units)
    
    def __radd__(self, other):
        return self.__add__(other)

In [30]:
test_addition_1 = test_mat_unit + 1.0
test_addition_2 = 1.0 + test_mat_unit
# test_addition_3 = test_mat_unit + u.m  # Should throw error, as it does
# test_addition_4 = test_mat_unit + test_mat_unit.unit  # Should throw error
test_addition_5 = test_mat_unit + test_mat_unit

test = MatrixWithUnits_simple(42, u.m)
test = MatrixWithUnits_simple([[42, 96]], [[u.m, u.m]])
# test = MatrixWithUnits_simple(np.array([[42, 96]]), u.m)
# test_addition_6 = 2.0 * u.m + test
test_addition_7 = test + 2.0 * u.m
# test_addition_8 = test + 2.0 * u.s  # Incompatible unit, should throw error, as it does

print(test_addition_1)
print(test_addition_2)
print(test_addition_5)
# print(test_addition_6)
print(test_addition_7)

array([[<Quantity 43.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 97. pc>]], dtype=object)
array([[<Quantity 43.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 97. pc>]], dtype=object)
array([[<Quantity 84.>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 192. pc>]], dtype=object)
array([[<Quantity 44. m>, <Quantity 98. m>]], dtype=object)


## astropy Stuff

In [31]:
from astropy.table import Table
from astropy.nddata import NDData

In [32]:
data = [[42 * u.s, 96 * u.m], [1 * u.kg, -1 * u.pc]]

test_table = Table(data)
# test_nddata = NDData(data)

NDData throws error...

In [33]:
print(test_table[0][0])
print(type(test_table[0][0]))

print(np.array(test_table))

print(test_table.values)

42.0 s
<class 'astropy.units.quantity.Quantity'>
[(<Quantity 42. s>, <Quantity 1. kg>)
 (<Quantity 96. m>, <Quantity -1. pc>)]
<bound method Table.values of <Table length=2>
 col0    col1 
object  object
------ -------
42.0 s  1.0 kg
96.0 m -1.0 pc>


In [54]:
print(np.array(test_table))
# print(2.0 * np.array(test_table))  # Error
# print(2.0 * test_table)
# print(1.0 + test_table)
print(test_table + test_table)

[(<Quantity 42. s>, <Quantity 1. kg>)
 (<Quantity 96. m>, <Quantity -1. pc>)]


TypeError: unsupported operand type(s) for +: 'Table' and 'Table'

## Fisher Class Tests

In [2]:
from lalsimulation.gwsignal.core import waveform as wfm
from gw_signal_tools.fisher_utils import (
    fisher_matrix, get_waveform_derivative_1D_with_convergence
)
from gw_signal_tools.fisher_matrix import FisherMatrix
from gw_signal_tools.inner_product import norm

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/lalsimulation/lalsimulation.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


The pyseobnr package has failed to load, you will not be able to employ SEOBNRv5 approximants.


In [3]:
f_min = 20.*u.Hz  # Cutoff frequency
f_max = 1024. * u.Hz  # Cutoff from PSD
# delta_f = 2**-6 * u.Hz  # Actually works much better
delta_f = 2**-8 * u.Hz  # Actually works much better
delta_t = 1.0/4096.0 * u.s
f_ref = 20.*u.Hz  # Frequency where we specify spins
# f_ref = f_min  # Frequency where we specify spins
distance = 1.*u.Mpc
inclination = 0.0*u.rad  # Value taken from posteriors.ipynb, where posterior of inclination is plotted
phiRef = 0.*u.rad
eccentricity = 0.*u.dimensionless_unscaled
longAscNodes = 0.*u.rad
meanPerAno = 0.*u.rad


# Define the approximant
approximant = 'IMRPhenomXPHM'
# approximant = 'SEOBNRv4'

# Initialize the generator
gen = wfm.LALCompactBinaryCoalescenceGenerator(approximant)

wf_center_parameters = {
    'total_mass': 100.*u.solMass,
    # 'mass_ratio': 0.5*u.dimensionless_unscaled,
    'sym_mass_ratio': 0.05*u.dimensionless_unscaled,
    'f22_start' : f_min,
    'f_max' : f_max,
    'deltaT' : delta_t,
    'deltaF' : delta_f,
    'f22_ref': f_ref,
    'phi_ref' : phiRef,
    'distance' : distance,
    'inclination' : inclination,
    'eccentricity' : eccentricity,
    'longAscNodes' : longAscNodes,
    'meanPerAno' : meanPerAno,
    'condition' : 0
    # 'condition' : 1  # Needs to be on for SEOB
}

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/lalsimulation/gwsignal/core/waveform.py:226: UserWarning: This code is currently UNREVIEWED, use with caution!
  warnings.warn("This code is currently UNREVIEWED, use with caution!")


In [4]:
fisher_val, info = fisher_matrix(
    wf_center_parameters,
    # 'total_mass',
    ['total_mass', 'distance'],
    gen,
    return_info=True
)

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/lalsimulation/gwsignal/core/utils.py:277: UserWarning: The larger object cannot be determined, assuming m1 >= m2.
  warnings.warn(warn_string)
XLAL Warning - XLALSimInspiralWaveformParamsLookupMass1 (LALSimInspiralWaveformParams.c:490): This code is currently UNREVIEWED, use with caution!
XLAL Warning - XLALSimInspiralCheckDeterminationOfMasses (LALSimInspiralWaveformParams.c:87): This code is currently UNREVIEWED, use with caution!
XLAL Warning - XLALSimInspiralWaveformParamsLookupTotalMass (LALSimInspiralWaveformParams.c:833): This code is currently UNREVIEWED, use with caution!
XLAL Warning - XLALSimInspiralWaveformParamsLookupSymMassRatio (LALSimInspiralWaveformParams.c:869): This code is currently UNREVIEWED, use with caution!
XLAL Warning - XLALSimInspiralGetMassRatioFromSymMassRatio (LALSimInspiralWaveformParams.c:427): This code is currently UNREVIEWED, use with caution!
XLAL Warning - XLALSimInspiralWavefo

In [5]:
print(fisher_val)
print(fisher_val.value)
print(fisher_val.unit)
print(fisher_val[0, 0].to(1/(u.Msun)**4))

array([[<Quantity 1.10504164e-76 6.397e-122 / kg4>,
        <Quantity -4.63382295e-41 1 / (Mpc solMass)>],
       [<Quantity -4.63382295e-41 1 / (Mpc solMass)>,
        <Quantity 1.93358056e-76 1.10306e-90 / m4>]],
      dtype=object)
[[ 1.10504164e-76 -4.63382295e-41]
 [-4.63382295e-41  1.93358056e-76]]
[[Unit("6.397e-122 / kg4") Unit("1 / (Mpc solMass)")]
 [Unit("1 / (Mpc solMass)") Unit("1.10306e-90 / m4")]]
1.1050416389760876e-76 1 / solMass4


In [1]:
# deriv, info = get_waveform_derivative_1D_with_convergence(
#     wf_center_parameters,
#     'total_mass',
#     gen,
#     return_info=True
# )

In [7]:
from gw_signal_tools import PLOT_STYLE_SHEET
plt.style.use(PLOT_STYLE_SHEET)

In [8]:
# plt.ion()

In [2]:
# # %matplotlib inline
# ax = info['convergence_plot']

# # plt.xlim(0, 42)
# plt.xlim(18, 60)

# # plt.show()
# ax.show()

In [10]:
fisher_matrix = FisherMatrix(
    wf_center_parameters,
    params_to_vary=['total_mass', 'distance'],
    wf_generator=gen,
    return_info=True
)

fisher_matrix.fisher = fisher_val

In [11]:
inv = fisher_matrix.inverse

In [13]:
print(inv)
print(inv.value)
print(inv.unit)

array([[<Quantity -90049.93381122 1.56323e+121 kg4>,
        <Quantity -2.15804533e+40 Mpc solMass>],
       [<Quantity -2.15804533e+40 Mpc solMass>,
        <Quantity -51463.55349704 9.06572e+89 m4>]],
      dtype=object)
[[-9.00499338e+04 -2.15804533e+40]
 [-2.15804533e+40 -5.14635535e+04]]
[[Unit("1.56323e+121 kg4") Unit("Mpc solMass")]
 [Unit("Mpc solMass") Unit("9.06572e+89 m4")]]
